In [2]:
import pandas as pd

The data frames `Customers`, `Employees`, `Offices`, `OrderDetails`, `Orders`, `Payments`, `ProductLines`, and `Products` contain data of the corresponding tables in the [ClassicModels database](https://www.richardtwatson.com/dm6e/Reader/ClassicModels.html).

The entity relationship diagram is shown here ![ERD](figures/ClassicModels.png)

Using Pandas merge and join operations, answer the following questions:

*One to many relationship*

- Report the account representative for each customer.
- Report total payments for Atelier graphique.
- Report the total payments by date
- Report the products that have not been sold.
- List the amount paid by each customer.
- How many orders have been placed by Herkku Gifts?
- Who are the employees in Boston?
- Report those payments greater than \\$100,000. Sort the report so the customer who made the highest payment appears first.
- List the value of 'On Hold' orders.
- Report the number of orders 'On Hold' for each customer.

*Many to many relationship*

- List products sold by order date.
- List the order dates in descending order for orders for the 1940 Ford Pickup Truck.
- List the names of customers and their corresponding order number where a particular order from that customer has a value greater than $25,000?
- Are there any products that appear on all orders?
- List the names of products sold at less than 80% of the MSRP.
- Reports those products that have been sold with a markup of 100% or more (i.e.,  the priceEach is at least twice the buyPrice)
- List the products ordered on a Monday.
- What is the quantity on hand for products listed on 'On Hold' orders?

In [3]:
Customers = pd.read_csv('data/ClassicModels_Customers.csv', sep=';')

In [4]:
Customers.head(1)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,customerLocation
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",NaN,Nantes,NaN,44000,France,1370,21000.0,0


In [5]:
Employees = pd.read_csv('data/ClassicModels_Employees.csv', sep=';')

In [6]:
Employees.head(1)

,employeeNumber,lastName,firstName,extension,email,reportsTo,jobTitle,officeCode
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,0,President,1


In [7]:
Offices = pd.read_csv('data/ClassicModels_Offices.csv', sep=';')

In [8]:
Offices.head(1)

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory,officeLocation
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NaN,0


In [9]:
OrderDetails = pd.read_csv('data/ClassicModels_OrderDetails.csv', sep=';')

In [10]:
OrderDetails.head(1)

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10107,S10_1678,30,81.35,2


In [11]:
Orders = pd.read_csv('data/ClassicModels_Orders.csv', sep=';')

In [12]:
Orders.head(1)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06 00:00:00,2003-01-13 00:00:00,2003-01-10 00:00:00,Shipped,NaN,363


In [13]:
Payments = pd.read_csv('data/ClassicModels_Payments.csv', sep=';')

In [14]:
Payments.head(1)

,checkNumber,paymentDate,amount,customerNumber
0,AB661578,2004-07-28 00:00:00,9415.13,471


In [15]:
ProductLines = pd.read_csv('data/ClassicModels_ProductLines.csv', sep=';')

In [16]:
ProductLines.head(1)

,productLine,textDescription,htmlDescription,image
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,NaN,NaN


In [17]:
Products = pd.read_csv('data/ClassicModels_Products.csv', sep=';')

In [18]:
Products.head(1)

,productCode,productName,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP,productLine
0,S10_1678,1969 Harley Davidson Ultimate Chopper,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.7,Motorcycles


In [19]:
# Report the account representative for each customer.

pd.merge(Customers, Employees, 
         left_on = 'salesRepEmployeeNumber', 
         right_on = 'employeeNumber', how='inner')[['customerName', 
                                                    'firstName', 'lastName']]


,customerName,firstName,lastName
0,Atelier graphique,Gerard,Hernandez
1,La Rochelle Gifts,Gerard,Hernandez
2,Euro+ Shopping Channel,Gerard,Hernandez
3,Daedalus Designs Imports,Gerard,Hernandez
4,Mini Caravy,Gerard,Hernandez
...,...,...,...
95,"Vida Sport, Ltd",Martin,Gerard
96,CAF Imports,Martin,Gerard
97,Precious Collectables,Martin,Gerard
98,"Corrida Auto Replicas, Ltd",Martin,Gerard


In [20]:
# Report total payments for Atelier graphique.

df_join = Customers.join(
    Orders, rsuffix = "_order").join(
    OrderDetails, rsuffix="_details")[Customers['customerName']=='Atelier graphique']

df_join['totalPrice'] = df_join['quantityOrdered'] * df_join['priceEach'] 
df_join['totalPrice'].sum()

2440.5

In [21]:
# Report the total payments by date

Payments.groupby('paymentDate')['amount'].sum()

paymentDate
2003-01-16 00:00:00    10223.83
2003-01-28 00:00:00    10549.01
2003-01-30 00:00:00     5494.78
2003-02-16 00:00:00    50218.95
2003-02-20 00:00:00    53959.21
                         ...   
2005-05-20 00:00:00    29070.38
2005-05-23 00:00:00    75020.13
2005-05-25 00:00:00    30253.75
2005-06-03 00:00:00    12432.32
2005-06-09 00:00:00    46656.94
Name: amount, Length: 232, dtype: float64

In [32]:
# Report the products that have not been sold.
#     same as products with no orders
df_join = pd.merge(Products, OrderDetails, how='left')
df_join[df_join['orderNumber'].isnull()]

,productCode,productName,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP,productLine,orderNumber,quantityOrdered,priceEach,orderLineNumber
1122,S18_3233,1985 Toyota Supra,1:18,Highway 66 Mini Classics,"This model features soft rubber tires, working...",7733,57.01,107.57,Classic Cars,NaN,NaN,NaN,NaN


In [34]:
# List the amount paid by each customer.
pd.merge(Payments, Customers).groupby('customerName')['amount'].sum()

customerName
AV Stores, Co.                 148410.09
Alpha Cognac                    60483.36
Amica Models & Co.              82223.23
Anna's Decorations, Ltd        137034.22
Atelier graphique               22314.36
                                 ...    
Vida Sport, Ltd                108777.92
Vitachrome Inc.                 72497.64
Volvo Model Replicas, Co        43680.65
West Coast Collectables Co.     43748.72
giftsbymail.co.uk               71783.75
Name: amount, Length: 98, dtype: float64

In [37]:
# How many orders have been placed by Herkku Gifts?

df_join = pd.merge(Customers, Orders)
df_join[df_join['customerName']=='Herkku Gifts'].count()['orderNumber']

3

In [42]:
# Who are the employees in Boston?
df_join = pd.merge(Employees, Offices)
df_join[df_join['city']=='Boston']

,employeeNumber,lastName,firstName,extension,email,reportsTo,jobTitle,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory,officeLocation
15,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,1143,Sales Rep,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NaN,0
16,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,1143,Sales Rep,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NaN,0


In [45]:
# Report those payments greater than $100,000. Sort the report so the customer who made the highest payment appears first.
pd.merge(Customers, Payments[Payments['amount'] > 100000]).sort_values(by='amount', ascending = False)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,customerLocation,checkNumber,paymentDate,amount
3,141,Euro+ Shopping Channel,Freyre,Diego,(91) 555 94 44,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,1370,227600.0,0,JE105477,2005-03-18 00:00:00,120166.58
2,141,Euro+ Shopping Channel,Freyre,Diego,(91) 555 94 44,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,1370,227600.0,0,ID10962,2004-12-31 00:00:00,116208.40
1,124,Mini Gifts Distributors Ltd.,Nelson,Susan,4155551450,5677 Strong St.,NaN,San Rafael,CA,97562,USA,1165,210500.0,0,KI131716,2003-08-15 00:00:00,111654.40
4,148,"Dragon Souveniers, Ltd.",Natividad,Eric,+65 221 7555,Bronz Sok.,Bronz Apt. 3/6 Tesvikiye,Singapore,NaN,079903,Singapore,1621,103800.0,0,KM172879,2003-12-26 00:00:00,105743.00
0,124,Mini Gifts Distributors Ltd.,Nelson,Susan,4155551450,5677 Strong St.,NaN,San Rafael,CA,97562,USA,1165,210500.0,0,AE215433,2005-03-05 00:00:00,101244.59


In [47]:
# List the value of 'On Hold' orders.
df_join = pd.merge(Orders[Orders['status']=='On Hold'], OrderDetails)
df_join['total'] = df_join['priceEach'] * df_join['quantityOrdered']
df_join['total'].sum()

169575.61000000004

In [49]:
# Report the number of orders 'On Hold' for each customer
pd.merge(Orders[Orders['status']=='On Hold'], Customers).groupby('customerName')['orderNumber'].count()

customerName
Gifts4AllAges.com            1
Tekni Collectables Inc.      1
The Sharp Gifts Warehouse    1
Volvo Model Replicas, Co     1
Name: orderNumber, dtype: int64

In [51]:
# List products sold by order date
pd.merge(pd.merge(Orders, OrderDetails), Products).sort_values(by='orderDate')

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP,productLine
0,10100,2003-01-06 00:00:00,2003-01-13 00:00:00,2003-01-10 00:00:00,Shipped,NaN,363,S18_1749,30,136.00,3,1917 Grand Touring Sedan,1:18,Welly Diecast Productions,This 1:18 scale replica of the 1917 Grand Tour...,2724,86.70,170.00,Vintage Cars
50,10100,2003-01-06 00:00:00,2003-01-13 00:00:00,2003-01-10 00:00:00,Shipped,NaN,363,S18_4409,22,75.46,4,1932 Alfa Romeo 8C2300 Spider Sport,1:18,Exoto Designs,This 1:18 scale precision die cast replica fea...,6553,43.26,92.03,Vintage Cars
75,10100,2003-01-06 00:00:00,2003-01-13 00:00:00,2003-01-10 00:00:00,Shipped,NaN,363,S24_3969,49,35.29,1,1936 Mercedes Benz 500k Roadster,1:24,Red Start Diecast,This model features grille-mounted chrome horn...,2081,21.75,41.03,Vintage Cars
25,10100,2003-01-06 00:00:00,2003-01-13 00:00:00,2003-01-10 00:00:00,Shipped,NaN,363,S18_2248,50,55.09,2,1911 Ford Town Car,1:18,Motor City Art Classics,"Features opening hood, opening doors, opening ...",540,33.30,60.54,Vintage Cars
128,10101,2003-01-09 00:00:00,2003-01-18 00:00:00,2003-01-11 00:00:00,Shipped,Check on availability.,128,S18_2795,26,167.06,1,1928 Mercedes-Benz SSK,1:18,Gearbox Collectibles,This 1:18 replica features grille-mounted chro...,548,72.56,168.75,Vintage Cars
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,10425,2005-05-31 00:00:00,2005-06-07 00:00:00,NaN,In Process,NaN,119,S32_1268,41,83.79,11,1980s GM Manhattan Express,1:32,Motor City Art Classics,This 1980s era new look Manhattan express is s...,5099,53.93,96.31,Trucks and Buses
687,10424,2005-05-31 00:00:00,2005-06-08 00:00:00,NaN,In Process,NaN,141,S32_3522,44,54.94,2,1996 Peterbilt 379 Stake Bed with Outrigger,1:32,Red Start Diecast,"This model features, opening doors, detailed e...",814,33.61,64.64,Trucks and Buses
962,10425,2005-05-31 00:00:00,2005-06-07 00:00:00,NaN,In Process,NaN,119,S24_2840,31,31.82,5,1958 Chevy Corvette Limited Edition,1:24,Carousel DieCast Legends,The operating parts of this 1958 Chevy Corvett...,2542,15.91,35.36,Classic Cars
1072,10425,2005-05-31 00:00:00,2005-06-07 00:00:00,NaN,In Process,NaN,119,S50_1392,18,94.92,2,Diamond T620 Semi-Skirted Tanker,1:50,Highway 66 Mini Classics,This limited edition model is licensed and per...,1016,68.29,115.75,Trucks and Buses


In [ ]:
# List the order dates in descending order for orders for the 1940 Ford Pickup Truck
